In [1]:
!pip install yahoo-fin
import pandas as pd 
from yahoo_fin import stock_info as si 
import numpy as np
import requests
#from pandas_datareader import DataReader

In [4]:
#tickers = si.tickers_sp500()
#tickers = ['AAPL','MSFT','AMZN','FB','GOOGL','GOOG','TSLA','JPM','JNJ']
tickers = ['FB']
recommendations = []

In [5]:
for ticker in tickers:
    lhs_url = 'https://query2.finance.yahoo.com/v10/finance/quoteSummary/'
    rhs_url = '?formatted=true&crumb=swg7qs5y9UP&lang=en-US&region=US&' \
              'modules=upgradeDowngradeHistory,recommendationTrend,' \
              'financialData,earningsHistory,earningsTrend,industryTrend&' \
              'corsDomain=finance.yahoo.com'
              
    url =  lhs_url + ticker + rhs_url
    r = requests.get(url)
    if not r.ok:
        recommendation = 6
    try:
        result = r.json()['quoteSummary']['result'][0]
        recommendation =result['financialData']['recommendationMean']['fmt']
    except:
        recommendation = 6
    
    recommendations.append(recommendation)

In [6]:
df = pd.DataFrame(list(zip(tickers, recommendations)), columns =['Stock', 'Score']) 
df = df.set_index('Stock')
df['Score'] = df['Score'].astype(float, errors = 'raise')
df
# 1 is Strong Buy, 2 is Buy, 3 is Hold, 4 is Sell, 5 is Hard Sell

,Score
Stock,
FB,1.8


In [7]:
def stock_score(row):
    if row['Score'] <= 2.0:
        signal = 1
    elif row['Score'] >= 4.0:
        signal = -1
    else:
        signal = 0     
    return signal

df['Signal'] = df.apply(stock_score, axis=1)
df['Action'] = ["Strong Buy" if x <= 1.5 else "Buy" if x <= 2.5 else "Hold" if x < 3.5 else "Hard Sell" if x >= 4.5 else "Sell" for x in df['Score']]
df

,Score,Signal,Action
Stock,,,
FB,1.8,1,Buy


In [8]:
#df2.to_csv('yf_signal_action.csv')